SETUP

In [ ]:
%pip install transformers
%pip install accelerate
%pip install safetensors
%pip install invisible-watermark>=0.2.0
%pip install omegaconf

In [ ]:
%pip install git+https://github.com/huggingface/diffusers.git@main

MODEL

In [ ]:
!wget https://civitai.com/api/download/models/127055/ -O anime_art_xl.safetensors

In [ ]:
!wget https://civitai.com/api/download/models/94057 -o fastnegativeV2.pt

SCRIPT

In [ ]:
from diffusers import StableDiffusionXLPipeline, UniPCMultistepScheduler,DiffusionPipeline,StableDiffusionPipeline
import torch

base = StableDiffusionXLPipeline.from_single_file(
    "./anime_art_xl.safetensors", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)

base.to("cuda")


refiner = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    text_encoder_2=base.text_encoder_2,
    vae=base.vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)
refiner.to("cuda")

In [ ]:
n_steps = 50
high_noise_frac = 0.75

In [ ]:
prompt = f'''
an 3D render of a pokemon ((mix between turtle and plant)) with ((red mark on feet)) and ((leaf crown)) on head,((side view)),kawaii,chibi,1pc,fully visible
'''
neg_prompt =f'''
cartoon, anime, sketches,
(worst quality, low quality:1.2), 
(deformed, distorted, disfigured, bad eyes), 
wrong nose, weird mouth, strange ears, bad anatomy, wrong anatomy,
amputation, extra limb, missing limb, floating limbs, 
(bad teeth, mutated hands, wrong fingers:1.2),
disconnected limbs, mutation, ugly, disgusting, (negative_hand-neg, bad_pictures:1.2),out of frame

'''
image = base(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_end=high_noise_frac,
    output_type="latent",
    negative_prompt=neg_prompt
).images
image = refiner(
    prompt=prompt,
    num_inference_steps=n_steps,
    denoising_start=high_noise_frac,
    image=image,
    negative_prompt=neg_prompt
).images[0]
image.show()

In [ ]:
name = "turtle_plant"
image.save(f"/notebooks/threestudio/load/images/{name}.png")